# 3: Test Agreement -- Normal Data

In [1]:
import pandas as pd
import numpy as np
import pickle
import time

from collections import Counter

from copy import deepcopy

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

# For keras dependencise
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras import backend as K

# For LRP Visuals
from deepexplain.tensorflow import DeepExplain
from keras.models import Model

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


## Load Data Etc.

In [2]:
# Pick a value of k for the dataset
k = 1

In [3]:
# # load the models from disk
model = load_model("NN.h5")

In [4]:
# Load Data
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

# Load the feature activations
X_train_act = np.load("X_train_act.npy")
X_test_act = np.load("X_test_act.npy")

# Load DeepLIFT contributions
X_train_cont = np.load("new_X_train_deeplift.npy")
X_test_cont = np.load("new_X_test_deeplift.npy")

## Contributions Search

In [5]:
nn_pred = model.predict_classes(X_test)

In [6]:
X_train_knn = X_train.reshape(X_train.shape[0], 28*28)
X_test_knn = X_test.reshape(X_test.shape[0], 28*28)

In [7]:
print("DeepLIFT-KNN:", X_train_cont.shape)
print("Activations:", X_train_act.shape)
print("Training:", X_train.shape)
print("Training k-NN:", X_train_knn.shape)
print("y_test:", y_test.shape)

DeepLIFT-KNN: (60000, 128)
Activations: (60000, 128)
Training: (60000, 1, 28, 28)
Training k-NN: (60000, 784)
y_test: (10000,)


In [8]:
techniques = [
    ["k-NN*", X_train_act, X_test_act],
    ["C-DeepLIFT", X_train_cont, X_test_cont]
             ]

for item in techniques:
    technique = item[0]
    train = item[1]
    test = item[2]

    print("==================================================")
    print(technique)
    print("==================================================")

    kNN = KNeighborsClassifier(n_neighbors=3, algorithm="brute") 
    kNN.fit(train, y_train)

    knn_predictions_test = kNN.predict(test)
    print("Accuracy:", accuracy_score(y_test, knn_predictions_test))
    print(confusion_matrix(y_test, knn_predictions_test, labels=None, sample_weight=None))
    print(" ")
    
    # What's the % that's different?
    correct = 0
    for i in range(len(nn_pred)):
        if knn_predictions_test[i] == nn_pred[i]:
            correct += 1
    print("Agreement", correct/len(nn_pred))

k-NN*
Accuracy: 0.9932
[[ 978    0    0    0    0    0    0    1    1    0]
 [   0 1135    0    0    0    0    0    0    0    0]
 [   2    1 1025    0    0    0    0    3    1    0]
 [   0    0    1 1004    0    2    0    1    2    0]
 [   0    0    0    0  977    0    1    0    0    4]
 [   1    0    0    5    0  885    1    0    0    0]
 [   2    2    0    0    1    2  951    0    0    0]
 [   0    4    4    0    0    0    0 1017    1    2]
 [   2    0    2    1    0    1    0    1  965    2]
 [   0    2    0    1    5    2    0    3    1  995]]
 
Agreement 0.9944
C-DeepLIFT
Accuracy: 0.9937
[[ 978    0    0    0    0    0    0    1    1    0]
 [   0 1134    0    1    0    0    0    0    0    0]
 [   0    1 1030    0    0    0    0    1    0    0]
 [   0    0    0 1007    0    2    0    0    1    0]
 [   0    0    0    0  978    0    1    0    0    3]
 [   1    0    0    6    0  882    1    0    1    1]
 [   3    2    1    0    1    4  947    0    0    0]
 [   0    4    6    0    0  